In [31]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50


In [32]:
img_size = 224

In [33]:
def conv_block(input, num_filters):
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(num_filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = layers.MaxPooling2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = layers.concatenate([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = layers.Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # Bridge
    b1 = conv_block(p4, 1024)
    # b1 = conv_block(p4, 2048)

    # Decoder
    # d1 = decoder_block(b1, s4, 1024)
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    # Output
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs)
    return model

input_shape = (img_size, img_size, 3)  # Adjust size and channels according to your dataset
model = build_unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [34]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import cv2
import PIL
from PIL import Image
# Paths to your data.
images_path = './archive/crack_segmentation_dataset/train/images/'
masks_path = './archive/crack_segmentation_dataset/train/masks/'


# Load and preprocess data
def load_data(images_path, masks_path, img_size):
    images = []  # List to store the images
    masks = []   # List to store the masks
    path = ''
    count = 0
    print(images_path)
    for filename in os.listdir(images_path):
        # if count > 500:
        #     break
        img_path = images_path + filename
        mask_path = masks_path + filename  # Assuming mask has same filename
        # print(img_path, "Sagrfgh")
        # print(img_path)
        # Load and preprocess the image
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        # print(img.size())
    #   if not img or not mask:
    #         continue
        img = cv2.resize(img, img_size) / 255.0

        # Load and preprocess the mask using OpenCV
        
        mask = cv2.resize(mask, img_size) / 255.0

        images.append(img)
        masks.append(mask)
        count += 1
    
    return np.array(images), np.array(masks)
images, masks = load_data(images_path, masks_path, (img_size, img_size))

./archive/crack_segmentation_dataset/train/images/


In [35]:
images.shape

(9603, 224, 224, 3)

In [36]:
images1, masks1 = images, masks

In [37]:
masks1 = np.expand_dims(masks1, axis=-1)

In [38]:
masks1.shape

(9603, 224, 224, 1)

In [39]:

X_train, X_val, y_train, y_val = train_test_split(images1, masks1, test_size=0.2)
X_train = X_train.reshape((-1, img_size, img_size, 3))
X_val = X_val.reshape((-1, img_size, img_size, 3))
y_train = y_train.reshape((-1, img_size, img_size, 1))
y_val = y_val.reshape((-1, img_size, img_size, 1))
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


X_train shape: (7682, 224, 224, 3)
y_train shape: (7682, 224, 224, 1)
X_val shape: (1921, 224, 224, 3)
y_val shape: (1921, 224, 224, 1)


In [40]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create data generators with desired augmentation parameters
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator()  # No augmentation for validation data

# Generate batches of augmented data from arrays (X_train and y_train)
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

# Train the model using the fit_generator function
history = model.fit(
    train_generator, 
    steps_per_epoch=len(X_train) // 32,  # Number of batches per epoch
    epochs=10, 
    validation_data=val_generator,
    validation_steps=len(X_val) // 32  # Number of batches for validation
)


Epoch 1/10
 34/240 ━━━━━━━━━━━━━━━━━━━━ 2:09:36 38s/step - accuracy: 0.9180 - loss: 0.2939

In [ ]:
# Load and preprocess the image
image_path = 'CFD_001.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (112, 112))  # Resize to match your model's input shape
# image = image / 255.0  # Normalize the pixel values
cv2.imwrite( "asg.jpg", image)
# Convert the image to a NumPy array with the expected shape
# input_image = np.expand_dims(image, axis=0)  # Add a batch dimension
prediction = model.predict(input_image)
prediction

NameError: name 'input_image' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

prediction *= 255

prediction_2d = prediction.squeeze()

plt.figure(figsize=(8, 8))
plt.imshow(prediction_2d, cmap='gray')
plt.colorbar()
plt.axis('off')
plt.show()

NameError: name 'prediction' is not defined

In [ ]:
import cv2
import numpy as np
def gaussianNoise(image):
    row, col, ch = image.shape
    mean = 0
    var = 0.5
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch).astype('uint8')
    noisy_image = cv2.add(image, gauss)
    return noisy_image
def gaussianBlur(image, kernel_size=(5, 5)):
    blurred_image = cv2.GaussianBlur(image, kernel_size, 0)
    return blurred_image
def SobelFilter(image):
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.sqrt(cv2.addWeighted(cv2.pow(sobelx, 2), 0.5, cv2.pow(sobely, 2), 0.5, 0))
    sobel = np.uint8(sobel)
    noisy_sobel = gaussianNoise(sobel)
    return noisy_sobel

def colorDistortion(image, brightness=1, contrast=1):
    # Clip the brightness and contrast values to prevent overflows
    brightness = np.clip(brightness, -100, 100)
    contrast = np.clip(contrast, -100, 100)

    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow) / 255
        gamma_b = shadow
        
        buf = cv2.addWeighted(image, alpha_b, image, 0, gamma_b)
    else:
        buf = image.copy()

    if contrast != 0:
        f = 131 * (contrast + 127) / (127 * (131 - contrast))
        alpha_c = f
        gamma_c = 127 * (1 - f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf


In [ ]:
image_path = 'CFD_001.jpg'
image = cv2.imread(image_path)
# image = cv2.resize(image, (112, 112))  # Resize to match your model's input shape
# image = image / 255.0  # Normalize the pixel values
cv2.imwrite( "GaussianNoise.jpg", gaussianNoise(image))
cv2.imwrite( "GaussianBlur.jpg", gaussianBlur(image))
cv2.imwrite( "SobelFilter.jpg", SobelFilter(image))
cv2.imwrite( "colorDistortion.jpg", colorDistortion(image))

 

True

In [1]:
from tensorflow.keras import layers, models

# Residual block
def residual_block(input_tensor, num_filters):
    x = layers.Conv2D(num_filters, 3, padding="same")(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(num_filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Add the input_tensor (residual connection) to the output of the convolution block
    x = layers.add([x, input_tensor])
    x = layers.ReLU()(x)

    return x

# Attention gate
def attention_gate(input_tensor, gate_tensor, num_filters):
    # Reshape gate tensor to match the input_tensor shape
    gate_resized = layers.Conv2D(num_filters, 1, padding='same')(gate_tensor)
    gate_resized = layers.BatchNormalization()(gate_resized)

    # Add the gate to the input_tensor
    x = layers.add([input_tensor, gate_resized])
    x = layers.Activation('relu')(x)

    # Sigmoid activation to get the attention coefficients
    attention = layers.Conv2D(1, 1, padding='same', activation='sigmoid')(x)
    x = layers.multiply([input_tensor, attention])

    return x

def conv_block(input_tensor, num_filters):
    # Modify the conv_block to include a residual structure
    x = residual_block(input_tensor, num_filters)
    return x

# Modify the encoder_block to return the residual block output
def encoder_block(input_tensor, num_filters):
    x = conv_block(input_tensor, num_filters)
    p = layers.MaxPooling2D((2, 2))(x)
    return x, p

# Modify the decoder_block to include an attention gate
def decoder_block(input_tensor, skip_features, num_filters):
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input_tensor)

    # Apply attention gate to the skip features before concatenating
    attention_skipped = attention_gate(skip_features, x, num_filters)
    
    x = layers.concatenate([x, attention_skipped])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = layers.Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # Bridge
    b1 = conv_block(p4, 1024)

    # Decoder
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    # Output
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = models.Model(inputs, outputs)
    return model

# Define your input shape
input_shape = (128, 128, 3)  # Adjust size and channels according to your dataset
model = build_unet(input_shape)


ValueError: Inputs have incompatible shapes. Received shapes (128, 128, 64) and (128, 128, 3)